<a href="https://colab.research.google.com/github/rimshasohai/10pearlstask/blob/main/10pearlstask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import requests
import pandas as pd
from datetime import datetime
import os

# Replace with your real tokens
AQICN_TOKEN = "2eb7e59a27fec6826892da8d9b3f9ff95e6c573e"
OWM_TOKEN = "ee88310481e99a7843bdedc0cca27fd1"

# URLs
aqicn_url = f"https://api.waqi.info/feed/karachi/?token={AQICN_TOKEN}"
owm_url = f"https://api.openweathermap.org/data/2.5/weather?q=Karachi&appid={OWM_TOKEN}&units=metric"

# Fetch data
aqi_data = requests.get(aqicn_url).json()
weather_data = requests.get(owm_url).json()

now = datetime.now()

if aqi_data.get("status") == "ok":
    iaqi = aqi_data["data"].get("iaqi", {})
    weather_main = weather_data.get("main", {})
    weather_wind = weather_data.get("wind", {})

    # Safely extract pollutants (may be missing!)
    def get_value(pollutant):
        return iaqi.get(pollutant, {}).get("v")

    row = {
        "timestamp": now.strftime("%Y-%m-%d %H:%M:%S"),
        "aqi": aqi_data["data"].get("aqi"),
        "pm25": get_value("pm25"),
        "pm10": get_value("pm10"),
        "co": get_value("co"),
        "no2": get_value("no2"),
        "so2": get_value("so2"),
        "o3": get_value("o3"),
        "temp": weather_main.get("temp"),
        "humidity": weather_main.get("humidity"),
        "pressure": weather_main.get("pressure"),
        "wind_speed": weather_wind.get("speed"),
        "hour": now.hour,
        "day": now.day,
        "month": now.month
    }

    df = pd.DataFrame([row])
    file = "karachi_realtime_aqi.csv"
    df.to_csv(file, mode='a', index=False, header=not os.path.exists(file))
    print("✅ Row added at", now)
else:
    print("❌ AQICN API error:", aqi_data)


✅ Row added at 2025-06-22 06:55:05.543668


In [22]:
df

,timestamp,aqi,pm25,pm10,co,no2,so2,o3,temp,humidity,pressure,wind_speed,hour,day,month
0,2025-06-22 06:55:05,161,161,None,None,None,None,None,31.9,66,999,6.17,6,22,6


In [24]:
import requests
import datetime

API_KEY = 'ee88310481e99a7843bdedc0cca27fd1'
lat, lon = 24.8607, 67.0011  # Karachi

url = f"https://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}"
resp = requests.get(url).json()

now = datetime.datetime.now()
pollution = resp['list'][0]['components']

data = {
    "timestamp": now.strftime("%Y-%m-%d %H:%M:%S"),
    "co": pollution.get("co"),
    "no2": pollution.get("no2"),
    "so2": pollution.get("so2"),
    "o3": pollution.get("o3"),
    "pm2_5": pollution.get("pm2_5"),
    "pm10": pollution.get("pm10"),
    "nh3": pollution.get("nh3")
}

print(data)


{'timestamp': '2025-06-22 06:56:30', 'co': 79.38, 'no2': 0.08, 'so2': 0.46, 'o3': 55.69, 'pm2_5': 30.76, 'pm10': 144.34, 'nh3': 0}


In [25]:
import requests
import pandas as pd
from datetime import datetime
import os
import time

# Your OpenWeatherMap API key
API_KEY = "ee88310481e99a7843bdedc0cca27fd1"
LAT = 24.8607  # Karachi latitude
LON = 67.0011  # Karachi longitude

# URLs
air_url = f"https://api.openweathermap.org/data/2.5/air_pollution?lat={LAT}&lon={LON}&appid={API_KEY}"
weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&appid={API_KEY}&units=metric"

# Fetch pollution and weather data
air_resp = requests.get(air_url).json()
weather_resp = requests.get(weather_url).json()

now = datetime.now()

try:
    air_data = air_resp["list"][0]["components"]
    weather_main = weather_resp.get("main", {})
    wind = weather_resp.get("wind", {})

    row = {
        "timestamp": now.strftime("%Y-%m-%d %H:%M:%S"),
        "aqi": air_resp["list"][0].get("main", {}).get("aqi"),
        "pm25": air_data.get("pm2_5"),
        "pm10": air_data.get("pm10"),
        "co": air_data.get("co"),
        "no2": air_data.get("no2"),
        "so2": air_data.get("so2"),
        "o3": air_data.get("o3"),
        "temp": weather_main.get("temp"),
        "humidity": weather_main.get("humidity"),
        "pressure": weather_main.get("pressure"),
        "wind_speed": wind.get("speed"),
        "hour": now.hour,
        "day": now.day,
        "month": now.month
    }

    df = pd.DataFrame([row])
    file = "karachi_realtime_aqi.csv"
    df.to_csv(file, mode='a', index=False, header=not os.path.exists(file))

    print(f"✅ Row added to {file} at {now.strftime('%Y-%m-%d %H:%M:%S')}")

except Exception as e:
    print("❌ Error:", str(e))


✅ Row added to karachi_realtime_aqi.csv at 2025-06-22 07:00:56
